### Agent Reasoning Loop

Instead of tool calling it in a single shot settings, the agent here is able to reason over tools and multiple steps.  
  
The function calling agent implementation integrates with the function calling capabilities of LLMs.  
  
These agents go beyond simply calling a tool in a single step; instead, they can reason through multiple steps and calls to functions, making decisions based on the results of each step. This involves leveraging the LLM's ability to understand the task, plan the necessary actions (or tool calls), and integrate the results across multiple steps.

In [1]:
! py -m pip install --upgrade python-dotenv
! py -m pip install --upgrade llama-index
! py -m pip install --upgrade llama-index-llms-openai
! py -m pip install --upgrade llama-index-embeddings-openai

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.1.0
    Uninstalling python-dotenv-1.1.0:
      Successfully uninstalled python-dotenv-1.1.0



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\RACHANAA\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ----------- ---------------------------- 2.1/7.6 MB 11.8 MB/s eta 0:00:01
   -------------------- ------------------- 3.9/7.6 MB 9.5 MB/s eta 0:00:01
   --------------------------------- ------ 6.3/7.6 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 7.6/7.6 MB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/812.0 kB ? eta -:--:--
   ------------ --------------------------- 262.1/812.0 kB ? eta -:--:--
   ---------------------------------------- 812.0/812.0 kB 2.2 MB/s 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.4.6 requires llama-index-core<0.13.0,>=0.12.18, but you have llama-index-core 0.13.3 which is incompatible.
llama-index-agent-openai 0.4.6 requires llama-index-llms-openai<0.4.0,>=0.3.0, but you have llama-index-llms-openai 0.5.4 which is incompatible.
llama-index-embeddings-gemini 0.3.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.13.3 which is incompatible.
llama-index-llms-gemini 0.4.14 requires llama-index-core<0.13.0,>=0.12.12, but you have llama-index-core 0.13.3 which is incompatible.
llama-index-multi-modal-llms-openai 0.4.3 requires llama-index-core<0.13.0,>=0.12.3, but you have llama-index-core 0.13.3 which is incompatible.
llama-index-multi-modal-llms-openai 0.4.3 requires llama-index-llms-openai<0.4.0,>=0.3.0, but you have llama-index-llms-openai


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\RACHANAA\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\RACHANAA\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [6]:
import os
import nest_asyncio

OPENAI_API_KEY = "sk-projcgA"
nest_asyncio.apply()

### Prerequisite

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()


In [8]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=5)

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [ ]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=10,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?",
)

In [ ]:
from typing import List
from llama_index.core.tools import FunctionTool  
from llama_index.core.vector_stores import FilterCondition

def vector_query(query: str, page_numbers: List[str]) -> str:
    """Perform a vector search over an index.
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    """
    
    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters = MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(
    name = "vector_tool",
    fn = vector_query
    )

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", 
    use_async=True
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine = summary_query_engine,
    description = (
        "Useful if you want to get a summary of MetaGPT"
    ),
)

#### Setup Function Calling Agent

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature = 0)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool],
    llm = llm,
    verbose = True
)

agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT,"
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT,and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Agent roles in MetaGPT"}
=== Function Output ===
The roles in MetaGPT include the Product Manager responsible for creating the Product Requirement Document (PRD) with goals, user stories, competitive analysis, and requirements. The Architect designs the system architecture and technical specifications based on the PRD. The Project Manager breaks down tasks and assigns them to Engineers for implementation. Engineers develop the code based on provided specifications, and the QA Engineer generates unit tests to ensure software quality. Each agent has a specific role in the collaborative workflow, contributing to the overall success of the project.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "How agents communicate with each other in MetaGPT"}
=== Function Output =

In [ ]:
print(response.source_nodes[0].get_content(metadata_mode = "all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-12-25
last_modified_date: 2024-12-25

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4, Jinlin Wang1, Zili Wang, Steven Ka Shing Yau5, Zijuan Lin4,
Liyang Zhou6, Chenyu Ran1, Lingfeng Xiao1,7, Chenglin Wu1†, J¨urgen Schmidhuber2,8
1DeepWisdom, 2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University, 4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University, 6University of Pennsylvania,
7University of California, Berkeley, 8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, while MBPP comprises 427 Python tasks covering core concepts and standard library features. The SoftwareDev dataset includes 70 representative examples of software development tasks with diverse scopes, such as mini-games, image processing algorithms, and data visualization. These datasets were utilized to evaluate the performance of MetaGPT in code generation tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, MBPP comprises 427 Python tasks, and the SoftwareDev dataset includ

In [ ]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "results over HumanEval dataset", "page_numbers": []}
=== Function Output ===
MetaGPT achieved a Pass rate of 85.9% and 87.7% over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved a Pass rate of 85.9% and 87.7% over the HumanEval dataset.


### Low level Debuggability and Control

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, the agent roles include the Product Manager, Architect, Project Manager, Engineers, and QA Engineer. The Product Manager is responsible for creating the Product Requirement Document (PRD), the Architect designs the system architecture, the Project Manager assigns tasks to Engineers, Engineers develop the code, and the QA Engineer ensures quality through unit tests and code reviews. Each agent has a specialized role in the software development process within the MetaGPT framework.


In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task ffaa2879-8edd-42cf-80a7-44f3bf7f85c4: 1
In MetaGPT, the agent roles include the Product Manager, Architect, Project Manager, Engineers, and QA Engineer. The Product Manager is responsible for creating the Product Requirement Document (PRD), the Architect designs the system architecture, the Project Manager assigns tasks to Engineers, Engineers develop the code, and the QA Engineer ensures quality through unit tests and code reviews. Each agent has a specialized role in the software development process within the MetaGPT framework.


In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task ffaa2879-8edd-42cf-80a7-44f3bf7f85c4: 1


TaskStep(task_id='ffaa2879-8edd-42cf-80a7-44f3bf7f85c4', step_id='4637ae18-5462-4333-a71f-25be13fe355d', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool with args: {"input": "How agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This allows them to exchange messages directly and subscribe to relevant messages based on their roles. The shared message pool and subscription mechanism enhance communication efficiency by providing a centralized platform for information exchange and ensuring that agents receive only task-related information, thus avoiding information overload.


In [ ]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, agents share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This allows agents to exchange messages directly and subscribe to relevant messages based on their roles. The shared message pool and subscription mechanism enhance communication efficiency by providing a centralized platform for information exchange and ensuring that agents receive only task-related information, thus avoiding information overload.
True


In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

In MetaGPT, agents share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This allows agents to exchange messages directly and subscribe to relevant messages based on their roles. The shared message pool and subscription mechanism enhance communication efficiency by providing a centralized platform for information exchange and ensuring that agents receive only task-related information, thus avoiding information overload.
